In [53]:
import pandas as pd 
from sqlalchemy import create_engine
import xlwings as xw

In [54]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [55]:
def load_file( path, sheet ) :
    book = xw.Book( path )
    df = book.sheets(sheet).used_range.options(pd.DataFrame).value
    return df

def sqlalchemy_connector( host, db, user, pw, sql ):
    engine = create_engine( 'mysql+pymysql://' + user + ':' + pw + '@' + host + ':3306/' + db )
    df = pd.read_sql_query( sql, engine )
    return df

In [56]:
event_df = load_file( 'C:/Users/10152638/Desktop/tmp/(221123) 강북_2022년_주요행사리스트 및 기지국리스트_11월4주~12월1주 행사리스트.xlsx', '상세리스트' )

In [57]:
event_df = event_df.reset_index() 

In [58]:
lte_site_raw = load_file( 'C:/Users/10152638/Desktop/tmp/LTE_NEW_SITE_INFO_2022-11-23.xlsb', 'LTE_NEW_SITE_INFO_2022-11-23' )

In [59]:
lte_site_raw = lte_site_raw.reset_index() 

In [60]:
num = range(0, 26)

In [61]:
col_list = [ 0, 1, 3, 4, 11, 12, 18 ]

In [62]:
col_list.append( 47 )

In [63]:
lte_site = lte_site_raw.iloc[ :, col_list ]

In [64]:
lte_site.head()

,LSM,DU번호,RRH_ID,RRH_NAME,CELL,CELL2,BW,ADD3
0,LSM1,SEC0006169,RNSL03820T,능동_광정빌딩_900M,0.0,-,900M,능동
1,LSM1,SEC0006169,RNSL03819T,군자동_비전빌딩_900M,0.0,-,900M,군자동
2,LSM1,SEC0006169,RNSL10439T,능동_길송빌딩_900M,11.0,-,900M,능동
3,LSM1,SEC0006169,RNSL03823T,중곡2_341-7번지_900M,11.0,-,900M,중곡동
4,LSM1,SEC0006169,RNSL03839T,중곡2_공영빌딩_900M,9.0,-,900M,중곡동


In [65]:
lte_site = lte_site.astype( str )

In [66]:
lte_site[ 'CELL' ] = lte_site[ 'CELL' ].replace( r'.0$', '', regex = True )
lte_site[ 'CELL2' ] = lte_site[ 'CELL2' ].replace( r'.0$', '', regex = True )
lte_site_cell = lte_site[ ( lte_site[ 'CELL2' ] == '' ) | ( lte_site[ 'CELL2' ] == '-' )]
lte_site_cell2 = lte_site[ ( lte_site[ 'CELL2' ] != '' ) & ( lte_site[ 'CELL2' ] != '-' ) ]
lte_site_cell[ 'CELL_CELL' ] = lte_site[ 'CELL' ]
lte_site_cell2[ 'CELL_CELL' ] = lte_site[ 'CELL' ] + '_' + lte_site[ 'CELL2' ]
slte_site = pd.concat( [ lte_site_cell, lte_site_cell2 ] )
lte_site[ 'CELL_CELL' ] = slte_site[ 'CELL_CELL' ]

C:\Users\10152638\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [67]:
slte_list = event_df[ ( event_df[ '망' ] == 'LTE' ) & ( event_df[ '벤더' ] == 'SS'  ) ]
slte_list = slte_list.reset_index( drop = True )

In [68]:
slte_list[ 'SECTOR(CELL)' ] = slte_list[ 'SECTOR(CELL)' ].astype( str )

In [69]:
prepro_cell = slte_list[ ~slte_list[ 'SECTOR(CELL)' ].str.contains( '_' ) ][ 'SECTOR(CELL)' ]
prepro_cell = prepro_cell.str.replace( r'.0$', '', regex = True )
prepro_cell2 = slte_list[ slte_list[ 'SECTOR(CELL)' ].str.contains( '_' ) ][ 'SECTOR(CELL)' ]

In [70]:
cell_result = pd.concat( [ prepro_cell, prepro_cell2 ], sort=False).sort_index()
slte_list[ 'SECTOR(CELL)' ] = cell_result

In [71]:
slte_check = pd.merge( slte_list, slte_site, left_on = '기지국ID', right_on = 'RRH_ID', how = 'inner' )

In [72]:
def slte_check_function( slte ) :
    name_wrong = slte[ ( slte[ '기지국명' ] != slte[ 'RRH_NAME' ] ) ]
    name_wrong[ '행사리스트' ] = name_wrong[ '기지국명' ]
    name_wrong[ 'SITE_INFO' ] = name_wrong[ 'RRH_NAME' ]
    name_wrong[ '오류발견' ] = '기지국명'
    name_wrong_idx = list( name_wrong.index )
    slte.loc[ name_wrong_idx, '기지국명' ] = slte.loc[ name_wrong_idx, 'RRH_NAME' ]
    

    lsm_wrong = slte[ ( slte[ '시스템 명' ] != slte[ 'LSM' ] ) ]
    lsm_wrong[ '행사리스트' ] = lsm_wrong[ '시스템 명' ]
    lsm_wrong[ 'SITE_INFO' ] = lsm_wrong[ 'LSM' ]
    lsm_wrong[ '오류발견' ] = '시스템명'
    lsm_wrong_idx = list( lsm_wrong.index )
    slte.loc[ lsm_wrong_idx, '시스템 명' ] = slte.loc[ lsm_wrong_idx, 'LSM' ]

    du_wrong = slte[ ( slte[ 'DU ID' ] != slte[ 'DU번호' ] ) ]
    du_wrong[ '행사리스트' ] = du_wrong[ 'DU ID' ]
    du_wrong[ 'SITE_INFO' ] = du_wrong[ 'DU번호' ]
    du_wrong[ '오류발견' ] = 'DU ID'
    du_wrong_idx = list( du_wrong.index )
    slte.loc[ du_wrong_idx, 'DU ID' ] = slte.loc[ du_wrong_idx, 'DU번호' ]


    bw_wrong = slte[ ( slte[ '5G/LTE 주파수' ] != slte[ 'BW' ] ) ]
    bw_wrong[ '행사리스트' ] = bw_wrong[ '5G/LTE 주파수' ]
    bw_wrong[ 'SITE_INFO' ] = bw_wrong[ 'BW' ]
    bw_wrong[ '오류발견' ] = '주파수'
    bw_wrong_idx = list( bw_wrong.index )
    slte.loc[ bw_wrong_idx, '5G/LTE 주파수' ] = slte.loc[ bw_wrong_idx, 'BW' ]

    cell_wrong = slte[ ( slte[ 'SECTOR(CELL)' ] != slte[ 'CELL_CELL' ] ) ]
    cell_wrong[ '행사리스트' ] = cell_wrong[ 'SECTOR(CELL)' ]
    cell_wrong[ 'SITE_INFO' ] = cell_wrong[ 'CELL_CELL' ]
    cell_wrong[ '오류발견' ] = 'SECTOR(CELL)'
    cell_wrong_idx = list( cell_wrong.index )
    slte.loc[ cell_wrong_idx, 'SECTOR(CELL)' ] = slte.loc[ cell_wrong_idx, 'CELL_CELL' ]

    result = pd.concat( [ name_wrong, lsm_wrong, du_wrong, bw_wrong, cell_wrong ] )

    num = list( range( 0, 19 ) )
    num.insert( 0, -3 )
    num.insert( 0, -2 )
    num.insert( 0, -1 )
    col_list = list( num )
    result = result.iloc[ :, col_list ] 
    slte = slte.iloc[ :, :19 ] 

    return result, slte

In [73]:
slte_wrong, slte_result = slte_check_function( slte_check )

#### 5G

In [74]:
nr_site_raw = load_file( 'C:/Users/10152638/Desktop/tmp/5G_NEW_SITE_INFO_2022-11-23.xlsb', '5G_NEW_SITE_INFO_2022-11-23' )
nr_site_raw = nr_site_raw.reset_index( drop = True )

In [75]:
nr_list = event_df[ ( event_df[ '망' ] == '5G' ) & ( event_df[ '벤더' ] == 'SS'  ) ]
nr_list = nr_list.reset_index( drop = True )

In [76]:
nr_list.head(2)

,팀,액세스망제어팀,행사명,기간,예상인원,망,벤더,시스템 명,ACPF NAME,RNC명\n(RNC 번호 2자리 표시),DU ID,SECTOR(CELL),5G/LTE 주파수,기지국ID,기지국명,서비스지역,증설유무\n(기존 OR 증설),통합NMS 운용현황,비고
0,청량엔지,강북,2022 남진 콘서트,2022-11-26 00:00:00,4000명,5G,SS,HH_CU04,NRSL64466S,ACPF_100004,None,NS00200176,18.0,3.5G,회기동_경희대경영대학관_32T_A,경희대학교 서울캠퍼스,기존,운용,None
1,청량엔지,강북,2022 남진 콘서트,2022-11-26 00:00:00,4000명,5G,SS,HH_CU04,NRSL65490S,ACPF_100004,None,NS00200178,5.0,3.5G,회기동_경희대경영대학관_32T_B,경희대학교 서울캠퍼스,기존,운용,None


In [77]:
col_list = [ 1, 2, 5, 7, 8, 13 ]

In [78]:
nr_site = nr_site_raw.iloc[ :, col_list ]

In [79]:
# prepro_cell = slte_list[ ~slte_list[ 'SECTOR(CELL)' ].str.contains( '_' ) ][ 'SECTOR(CELL)' ]
# prepro_cell = prepro_cell.str.replace( r'.0$', '', regex = True )
# prepro_cell2 = slte_list[ slte_list[ 'SECTOR(CELL)' ].str.contains( '_' ) ][ 'SECTOR(CELL)' ]

In [80]:
nr_site = nr_site.astype( str )
nr_site[ 'CELL_ID' ] = nr_site[ 'CELL_ID' ].str.replace( r'.0$', '', regex = True )

In [81]:
nr_list[ nr_list[ '기지국ID' ] == 'NS00200378' ]

,팀,액세스망제어팀,행사명,기간,예상인원,망,벤더,시스템 명,ACPF NAME,RNC명\n(RNC 번호 2자리 표시),DU ID,SECTOR(CELL),5G/LTE 주파수,기지국ID,기지국명,서비스지역,증설유무\n(기존 OR 증설),통합NMS 운용현황,비고


In [82]:
nr_site[ 'CELL_ID' ].unique()

array(['64', '66', '68', ..., '1749', '1751', '1989'], dtype=object)

In [83]:
nr_list = nr_list.astype( str )

In [84]:
nr_list[ 'SECTOR(CELL)' ] = nr_list[ 'SECTOR(CELL)' ].str.replace( r'.0$', '', regex = True )

In [85]:
nr_check = pd.merge( nr_list, nr_site, left_on = '기지국ID', right_on = 'RU_ID', how = 'inner' )

In [86]:
def nr_check_function( nr ) :
    name_wrong = nr[ ( nr[ '기지국명' ] != nr[ 'RU_NAME' ] ) ]
    name_wrong[ '행사리스트' ] = name_wrong[ '기지국명' ]
    name_wrong[ 'SITE_INFO' ] = name_wrong[ 'RU_NAME' ]
    name_wrong[ '오류발견' ] = '기지국명'
    name_wrong_idx = list( name_wrong.index )
    nr.loc[ name_wrong_idx, '기지국명' ] = nr.loc[ name_wrong_idx, 'RU_NAME' ]
    

    lsm_wrong = nr[ ( nr[ '시스템 명' ] != 'HH_USM' ) ]
    lsm_wrong[ '행사리스트' ] = lsm_wrong[ '시스템 명' ]
    lsm_wrong[ 'SITE_INFO' ] = 'HH_USM'
    lsm_wrong[ '오류발견' ] = '시스템명'
    lsm_wrong_idx = list( lsm_wrong.index )
    nr.loc[ lsm_wrong_idx, '시스템 명' ] = 'HH_USM'

    du_wrong = nr[ ( nr[ 'DU ID' ] != nr[ 'DU_ID' ] ) ]
    du_wrong[ '행사리스트' ] = du_wrong[ 'DU ID' ]
    du_wrong[ 'SITE_INFO' ] = du_wrong[ 'DU_ID' ]
    du_wrong[ '오류발견' ] = 'DU ID'
    du_wrong_idx = list( du_wrong.index )
    nr.loc[ du_wrong_idx, 'DU ID' ] = nr.loc[ du_wrong_idx, 'DU_ID' ]


    bw_wrong = nr[ ( nr[ '5G/LTE 주파수' ] != '3.5G' ) ]
    bw_wrong[ '행사리스트' ] = bw_wrong[ '5G/LTE 주파수' ]
    bw_wrong[ 'SITE_INFO' ] = '3.5G'
    bw_wrong[ '오류발견' ] = '주파수'
    bw_wrong_idx = list( bw_wrong.index )
    nr.loc[ bw_wrong_idx, '5G/LTE 주파수' ] = '3.5G'

    cell_wrong = nr[ ( nr[ 'SECTOR(CELL)' ] != nr[ 'CELL_ID' ] ) ]
    cell_wrong[ '행사리스트' ] = cell_wrong[ 'SECTOR(CELL)' ]
    cell_wrong[ 'SITE_INFO' ] = cell_wrong[ 'CELL_ID' ]
    cell_wrong[ '오류발견' ] = 'SECTOR(CELL)'
    cell_wrong_idx = list( cell_wrong.index )
    nr.loc[ cell_wrong_idx, 'SECTOR(CELL)' ] = nr.loc[ cell_wrong_idx, 'CELL_ID' ]

    result = pd.concat( [ name_wrong, lsm_wrong, du_wrong, bw_wrong, cell_wrong ] )
    num = list( range( 0, 19 ) )
    num.insert( 0, -3 )
    num.insert( 0, -2 )
    num.insert( 0, -1 )
    col_list = list( num )
    result = result.iloc[ :, col_list ] 
    nr = nr.iloc[ :, :-6 ] 

    return result, nr

In [87]:
nr_wrong, nr_result = nr_check_function( nr_check )

#### ELG

In [88]:
elte_site_raw = load_file( 'C:/Users/10152638/Desktop/tmp/ELG_LTE_SITE_INFO_20221123.xlsb', 'ELG_LTE_SITE_INFO_20221123' )
elte_site_raw = elte_site_raw.reset_index( drop = True )

In [89]:
elte_list = event_df[ ( event_df[ '망' ] == 'LTE' ) & ( event_df[ '벤더' ] == 'ELG'  ) ]
elte_list = elte_list.reset_index( drop = True )

In [90]:
col_list = list( [ 2, 3, 9, 10, 20 ] )
elte_site = elte_site_raw.iloc[ :, col_list ]
elte_site = elte_site.astype( str )
elte_site[ 'Cell' ] = elte_site[ 'Cell' ].str.replace( r'.0$', '', regex = True )

In [91]:
elte_site = elte_site.astype( str )

In [92]:
elte_site[ 'ENB' ] = elte_site[ 'EUtranCellFDD' ].str.split( '_' ).str.get( 0 )

In [93]:
elte_site = elte_site.groupby( [ 'ENB', 'nms장비ID', 'nms장비명' ] ).agg( { 'Cell' : '_'.join, 'DL주파수' : '_'.join } ).reset_index()

In [94]:
elte_site.groupby( [ 'ENB', 'nms장비ID', 'nms장비명' ] ).agg( { 'Cell' : '_'.join, 'DL주파수' : '_'.join } ).reset_index()

,ENB,nms장비ID,nms장비명,Cell,DL주파수
0,LGE0000021,RNKW04245G,인제상남T1(900M),89_89,900M_900M
1,LGE0000021,RQKW70133G,인제과학화훈련단_LRIU-T(30M),21_61,1.8G_1.8H
2,LGE0000021,RQKW70181G,인제_곤충바이오센터T,25_65,1.8G_1.8H
3,LGE0000021,RQKW70195G,인제푸르미아파트103동옥상T,26_66,1.8G_1.8H
4,LGE0000021,RRKW02040G,인제마의태자캠핑장T,28,1.8G
...,...,...,...,...,...
7642,LGE1000060,RQKW49512G,홍천화상대리아홉사리로펜션단지T,21_61,1.8G_1.8H
7643,LGE1000062,-,-,51,900M
7644,LGE1000062,RQKW49572G,인제과학화훈련단본청인빌딩T,24_64,1.8G_1.8H
7645,LGE1000062,RQKW49588G,인제과학화대항군연대T,23_63,1.8G_1.8H


In [95]:
elte_site_raw.head(1)

,eNBID,DU,EUtranCellFDD,Cell,Rel_Cell,Sector,Sec_Port,DU_Port,DU_RRU,nms장비ID,nms장비명,nms장비상태,Cell_Admin,RRU_Admin,ECI,nmsCellID,PCI,nmsPCI,rachRootSequence,TAC,DL주파수,earfcndl,earfcnul,userLabel,MIMO여부,Combined여부,Termpoint,로밍_DU,로밍_Cell,upperLayerInd,중요국소,EndcAllowedPlmn,eMTC,nms위도,nms경도,latitude,longitude,Scheduling,pdcchPowerBoostMax,pdcchCfiMode,dlChannelBandwidth,ulChannelBandwidth,ulFrequencyAllocationProportion,ulConfigurableFrequencyStart,qRxLevMin,qHyst,cellRange,noOfPucchCqiUsers,noOfPucchSrUsers,primaryPlmnReserved,qQualMin,noOfRxAntennas,noOfTxAntennas,dlPowerRatio,maximumTransmissionPower,dlTrafficDelay,ulTrafficDelay,dlAttenuation,ulAttenuation,configuredMaxTxPower,b1Threshold,crsGain,RE_Boosting,productionDate,productName,productNumber,productRevision,serialNumber,nmsDU랙번호,nmsDU번호,nmsCH카드,nmsCH포트,nms집중국,nms도로명주소,nms시도,nms시군구,nms읍면동,nms번지,nms건물,nms세부주소,nmsRU서비스지역,nms운용팀,nms담당자,SW집중국,S1_ip,SW_NAME,SW_port,Primary_SW_IP,Secondary_SW_IP,Primary_회선번호,Secondary_회선번호,RPS
0,35572.0,DU30,LGE0000021_1A,21.0,4508-35572-21,LGE0000021_S1A,LGE0000021_S1A_D_S1A_4_Direct,LGE0000021_D,LGE0000021_MRIR30-D-S1A,RQKW70133G,인제과학화훈련단_LRIU-T(30M),운용,UNLOCKED,UNLOCKED,9106453.0,9106453_9106493_9106533,375.0,375_375_375,400.0,1574.0,1.8G,1550.0,19550.0,LGE0000021_1A_RQKW70133G,MIMO,Normal,-,OFF,OFF,ON,-,5G_ON,True,37-52-27.391,128-15-34.060,37874104.0,128259429.0,:RESOURCE_FAIR:DELAY_BASED:RESOURCE_FAIR:RESOU...,6.0,CFI_AUTO_MAXIMUM_3,20000.0,20000.0,72.0,51.0,-128.0,2.0,15.0,720.0,720.0,False,0.0,2.0,2.0,100.0,490.0,76000.0,76000.0,-160.0,0.0,2000.0,38.0,0.0,Pb_0(RE_Boost_ON),20170804.0,Radio2203B3,KRC161488/1,R1E,D826163212,2.0,2.0,1.0,LGE0000021:상남R1(분기)-02-02,상남R1(분기),강원도인제군-,강원도,인제군,상남면상남리,1308-0,과학화전투훈련단대항군연대연대동,상남기지국,인빌딩,영동엔지니어링팀,김용해,상남R1(분기),10.74.82.98/29,Sangnam.R1-LAX001-2,13.0,10.74.81.1,10.74.81.2,12313012-1243,12313012-1244,OFF


In [96]:
elte_list[ 'SECTOR(CELL)' ] = elte_list[ 'SECTOR(CELL)' ].astype( str )

In [97]:
elte_list[ 'SECTOR(CELL)' ] = elte_list[ 'SECTOR(CELL)' ].str.replace( r'.0$', '', regex = True )
elte_check = pd.merge( elte_list, elte_site, left_on = '기지국ID', right_on = 'nms장비ID', how = 'inner' )

In [98]:
def elte_check_function( nr ) :
    name_wrong = nr[ ( nr[ '기지국명' ] != nr[ 'nms장비명' ] ) ]
    name_wrong[ '행사리스트' ] = name_wrong[ '기지국명' ]
    name_wrong[ 'SITE_INFO' ] = name_wrong[ 'nms장비명' ]
    name_wrong[ '오류발견' ] = '기지국명'
    name_wrong_idx = list( name_wrong.index )
    nr.loc[ name_wrong_idx, '기지국명' ] = nr.loc[ name_wrong_idx, 'nms장비명' ]
    


    lsm_wrong = nr[ ( nr[ '시스템 명' ].isin( [ 'HH_ENM' ] ) ) ]
    lsm_wrong[ '행사리스트' ] = lsm_wrong[ '시스템 명' ]
    lsm_wrong[ 'SITE_INFO' ] = 'HH_ENM'
    lsm_wrong[ '오류발견' ] = '시스템명'
    lsm_wrong_idx = list( lsm_wrong.index )
    nr.loc[ lsm_wrong_idx, '시스템 명' ] = 'HH_ENM'

    du_wrong = nr[ ( nr[ 'DU ID' ] != nr[ 'ENB' ] ) ]
    du_wrong[ '행사리스트' ] = du_wrong[ 'DU ID' ]
    du_wrong[ 'SITE_INFO' ] = du_wrong[ 'ENB' ]
    du_wrong[ '오류발견' ] = 'DU ID'
    du_wrong_idx = list( du_wrong.index )
    nr.loc[ du_wrong_idx, 'DU ID' ] = nr.loc[ du_wrong_idx, 'ENB' ]


    bw_wrong = nr[ ( nr[ '5G/LTE 주파수' ] != nr[ 'DL주파수' ] ) ]
    bw_wrong[ '행사리스트' ] = bw_wrong[ '5G/LTE 주파수' ]
    bw_wrong[ 'SITE_INFO' ] = nr[ 'DL주파수' ]
    bw_wrong[ '오류발견' ] = '주파수'
    bw_wrong_idx = list( bw_wrong.index )
    nr.loc[ bw_wrong_idx, '5G/LTE 주파수' ] = nr[ 'DL주파수' ]

    cell_wrong = nr[ ( nr[ 'SECTOR(CELL)' ] != nr[ 'Cell' ] ) ]
    cell_wrong[ '행사리스트' ] = cell_wrong[ 'SECTOR(CELL)' ]
    cell_wrong[ 'SITE_INFO' ] = cell_wrong[ 'Cell' ]
    cell_wrong[ '오류발견' ] = 'SECTOR(CELL)'
    cell_wrong_idx = list( cell_wrong.index )
    nr.loc[ cell_wrong_idx, 'SECTOR(CELL)' ] = nr.loc[ cell_wrong_idx, 'Cell' ]

    result = pd.concat( [ name_wrong, lsm_wrong, du_wrong, bw_wrong, cell_wrong ] )
    num = list( range( 0, 19 ) )
    num.insert( 0, -3 )
    num.insert( 0, -2 )
    num.insert( 0, -1 )
    col_list = list( num )
    result = result.iloc[ :, col_list ] 
    nr = nr.iloc[ :, :-5 ] 

    return result, nr

In [99]:
elte_wrong, elte_result = elte_check_function( elte_check )

In [100]:
wrong_result = pd.concat( [ slte_wrong, nr_wrong, elte_wrong ] )

In [101]:
# right_result =  pd.concat( [ slte_result, nr_result, elte_result ] )

In [102]:
# right_result.drop_duplicates( inplace = True )

In [103]:
wrong_result.to_excel( 'C:/Users/10152638/Desktop/tmp/행사리스트_검증.xlsx', encoding = 'euc-kr', index = False )

In [104]:
# right_result.to_excel( 'C:/Users/10152638/Desktop/tmp/행사리스트_수정.xlsx', encoding = 'euc-kr', index = False )

In [2]:
lists = [('5G',), ('SS_LTE',), ('TEMP',), ('information_schema',), ('mysql',), ('oneview',), ('performance_schema',), ('sys',), ('test',)]

In [3]:
db_list = []
for i in range( len( lists ) ) :
    db_list.append( lists[ i ][ 0 ] )

In [4]:
db_list

['5G',
 'SS_LTE',
 'TEMP',
 'information_schema',
 'mysql',
 'oneview',
 'performance_schema',
 'sys',
 'test']

In [ ]:
0